# Import Packages

In [2]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config
import json
from bs4 import BeautifulSoup
import colorama
from colorama import Fore

import pprint
import random
from time import sleep
import pickle
from IPython.display import IFrame
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import difflib
import warnings
warnings.filterwarnings('ignore')

## Authenticate Spotipy

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [4]:
def play_song(track_id):
    display(IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",))

# Load Song Collection, Train Scaler & Kmeans

In [5]:
df_song_collection = pd.read_csv(r"C:\Users\p.kollhof\Documents\IRONHACK_GitHub\Spotify_Song_Recommender\Model_DiversifiedPlaylistApproach_no_accousticness_no_speechiness_KickOutliersRecluster\11137_Song_Collection_noFliers_reCLUSTERED.csv")
df_song_collection_features = df_song_collection[['danceability', 'energy', 'loudness', 'liveness', 'valence', 
                                                 'tempo', 'acousticness']] # ,'speechiness', 'instrumentalness
X = df_song_collection_features
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

kmeans = KMeans(n_clusters = 8, random_state=1234)
kmeans.fit(X_scaled_df)

clusters = kmeans.predict(X_scaled_df)
X["clusters"] = clusters

## Prompt User Song Input, 

In [6]:
def get_user_song_input():
    user_song_input = input("Pick any song: ")
    return user_song_input

## Get Song Data and play embedded

In [7]:
def get_user_song_id():
    user_song_name =  get_user_song_input()
    user_song = sp.search(q=user_song_name,limit=10,market="DE")
    user_song_id = user_song["tracks"]["items"][0]["id"]
    return user_song_id

In [8]:
def get_user_song_id_given(user_song_name):
    user_song = sp.search(q=user_song_name,limit=10,market="DE")
    user_song_id = user_song["tracks"]["items"][0]["id"]
    return user_song_id

## Get Song Features

In [9]:
def get_user_song_audiofeatures(user_song_id):
    user_song_audiofeatures = sp.audio_features(user_song_id)
    df_user_input = pd.DataFrame(user_song_audiofeatures)
    df_user_input = df_user_input[['danceability', 'energy', 'loudness', 'liveness', 'valence', 
                                     'tempo', 'acousticness']] # ,'speechiness', 'instrumentalness
    return df_user_input

## Scale User Song

In [10]:
def scale_user_song(scaler, df):
    X_scaled_user = scaler.transform(df)
    X_scaled_user_df = pd.DataFrame(X_scaled_user, columns=df.columns)
    return X_scaled_user_df

## Return User Song Cluster

In [11]:
def get_user_song_cluster(scaled_df, kmeans):
    user_song_cluster = kmeans.predict(scaled_df)[0]
    return user_song_cluster

# Give Recommendation based on User Song Input

In [12]:
def give_recommendation_sample(df_song_collection, user_song_cluster, n):
    sample_df = df_song_collection[df_song_collection['cluster'] == user_song_cluster].sample(n)
    #sample_df = sample[['danceability', 'energy', 'loudness', 'liveness', 'valence', 
                     #'tempo', 'speechiness', 'acousticness', 'instrumentalness']]
    return sample_df

#def give_recommendation_text(sample, user_song_cluster, df_song_collection):
    #recom_song_name = list(sample["song_name"])
    #recom_song_artist = list(sample["song_artist"])
    #recom_song_id = list(sample["id"])
    #recommendation = list(zip(recom_song_name, recom_song_artist, recom_song_id))
    #return recommendation

def sort_recommendation_user_song_similarities():
    X_scaled_recom = scaler.transform(sample)
    X_scaled_recom_df = pd.DataFrame(X_scaled_recom, columns=sample.columns)
    
    X_scaled_user_df_features = list(X_scaled_user_df.values[0])#
    X_scaled_recom_df_features = np.array(X_scaled_recom_df).tolist()

    diff_matrix = [[]] * len(X_scaled_recom_df_features)

    for idx in range(len(diff_matrix)):
        for val in range(len(X_scaled_user_df_features)):
            diff_matrix[idx].append( X_scaled_user_df_features[val] + X_scaled_recom_df_features[idx][val] )

    diff_matrix = diff_matrix[0]
    diff_matrix = [diff_matrix[:7], diff_matrix[7:-7], diff_matrix[-7:]]
    diff_matrix_sums = [abs(sum(submatrix)) for submatrix in diff_matrix]
    diff_matrix_sums_min = diff_matrix_sums.index(min(diff_matrix_sums))
    diff_matrix_sums_max = diff_matrix_sums.index(max(diff_matrix_sums))
    diff_matrix_sums_min, diff_matrix_sums_max

    similarity_sorter = [diff_matrix_sums_min,
                        list(set(range(len(diff_matrix))) - set([diff_matrix_sums_min, diff_matrix_sums_max]))[0],
                        diff_matrix_sums_max]
    return similarity_sorter, X_scaled_recom_df

def play_sorted3_recommendations(sample):
    similarity_sorter = sort_recommendation_user_song_similarities()[0]
    recom_song_name = list(sample["song_name"])
    recom_song_artist = list(sample["song_artist"])
    recom_song_id = list(sample["id"])
    recommendation = list(zip(recom_song_name, recom_song_artist, recom_song_id))

    for recom in similarity_sorter:
        play_song(recommendation[recom][2])
        #print(f"You might also like '{recommendation[recom][0]}' by {recommendation[recom][1]}!")
        
def play_recommendations(sample_df):
    recom_song_name = list(sample_df["song_name"])
    recom_song_artist = list(sample_df["song_artist"])
    recom_song_id = list(sample_df["id"])
    recommendation = list(zip(recom_song_name, recom_song_artist, recom_song_id))

    for recom in range(len(recommendation)):
        play_song(recommendation[recom][2])
        #print(f"You might also like '{recommendation[recom][0]}' by {recommendation[recom][1]}!")

# Output Pipeline

################################################################################################################

In [35]:
user_song_id = get_user_song_id()
df_user_input = get_user_song_audiofeatures(user_song_id)

play_song(user_song_id)

Pick any song: lose yourself


In [36]:
X_scaled_user_df = scale_user_song(scaler, df_user_input)
user_song_cluster = get_user_song_cluster(X_scaled_user_df, kmeans)
sample_df = give_recommendation_sample(df_song_collection, user_song_cluster, 3)
play_recommendations(sample_df)
